<img src="https://github.com/Databricks-BR/lab_genai/blob/main/img/header.png?raw=true" width=100%>

# Lab 02 - Criando um Agente

LLMs são ótimos para responder a perguntas gerais. No entanto, isso sozinho não é suficiente para fornecer valor aos seus clientes.

Para ser capaz de fornecer respostas mais complexas, informações adicionais e específicas para o usuário são necessárias, como seu ID de contrato, o último e-mail que enviaram para o seu suporte ou seu relatório de compras mais recentes.

Agentes são projetados para superar este desafio. Eles são implantações de IA mais avançadas, compostas por múltiplas entidades (ferramentas) especializadas em diferentes ações (recuper informações ou interagir com sistemas externos).

De forma geral, você constrói e apresenta um conjunto de funções personalizadas para a IA. A LLM pode então raciocinar sobre quais informações precisam ser reunidas e quais ferramentas utilizar para responder às intruções recebidas.

<br><img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/llm-tools-functions-flow.png?raw=true" width="100%">

# Configuração do ambiente

Vamos começar selecionando o nosso banco de dados

In [0]:
%sql USE academy.playground

# Exercício 01 - Usando LLMs

## a. Usando Databricks Foundation Models

<img src="https://docs.databricks.com/en/_images/serving-endpoints-list.png" style="float: right; padding-left: 10px; padding-top: 15px" width=600>

Precisamos de um modelo capaz de interpretar o texto das avaliações e extrair as informações desejadas. Para isso, vamos utilizar **[Foundation Models](https://docs.databricks.com/en/machine-learning/foundation-models/index.html#pay-per-token-foundation-model-apis)**, que são grandes modelos de linguagem (LLMs) servidos pela Databricks e que podem ser consultados sob-demanda sem a necessidade de implantação ou gerenciamento desses recursos.

Alguns modelos disponíveis são:

- Meta Llama
- OpenAI GPT e GPT OSS
- Anthropic Claude Sonnet e Opus
- Google Gemini e Gemma
- GTE Large
- BGE Large

Agora, vamos vê-los em funcionamento!

1. No **menu principal** à esquerda, clique em **`Serving`**
2. No card do modelo **Llama 4 Maverick**, clique em **`Use`**
3. Adicione a instrução abaixo:
    ```
    Classifique o sentimento da seguinte avaliação:
    Comprei um tablet e estou muito insatisfeito com a qualidade da bateria. Ela dura muito pouco tempo e demora muito para carregar.
    ```
    <br>
4. Clique no ícone **enviar**

Com isso, já conseguimos de forma rápida começar a prototipar nossos novos produtos de dados!

**NOTA:** A lista completa de modelos pode ser acessada em: https://docs.databricks.com/aws/en/machine-learning/foundation-model-apis/supported-models

## b. Testando modelos no AI Playground

<img src="https://docs.databricks.com/en/_images/ai-playground.gif" style="float: right; padding-left: 10px" width=600>

Para decidir qual o melhor modelo e instrução para o nosso caso de uso, podemos utilizar o **[AI Playground](https://docs.databricks.com/en/large-language-models/ai-playground.html)**.

Assim, podemos testar rapidamente diversas combinações de modelos e instruções através de uma interface intuitiva e escolher a melhor opção par utilizarmos no nosso projeto.

Vamos fazer o seguinte teste:

1. No **menu principal** à esquerda, clique em **`Playgroud`**
2. Clique no **seletor de modelos** e selecione o modelo **`Llama 4 Maverick`**
3. Clique no ícone **`Add endpoint`**
4. Clique no **seletor de modelos** e selecione o modelo **`GPT OSS 20B`**
5. Clique no ícone **`Add endpoint`**
6. Clique no **seletor de modelos** e selecione o modelo **`Gemma 3 12B`**
7. Adicione a instrução abaixo:
    ```
    Classifique o sentimento da seguinte avaliação:
    Comprei um tablet e estou muito insatisfeito com a qualidade da bateria. Ela dura muito pouco tempo e demora muito para carregar.
    ```
    <br>
8. Clique no ícone **enviar**

Agora, podemos comparar as respostas, o tempo e o custo de cada um dos modelos para escolher aquele que melhor atende às necessidades do nosso projeto!

# Exercício 02 - Definindo as ferramentas

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/llm-tools-functions-playground.gif?raw=true" style="float: right; padding-left: 10px" width=600>

As ferramentas de agentes de IA permitem que os agentes realizem tarefas além da geração de linguagem, como recuperar dados estruturados ou não estruturados e executar código personalizado.

Para criar uma ferramenta com o Mosaic AI Agent Framework, você pode usar qualquer combinação dos seguintes métodos:

|Método| Descrição|
|---|---|
|**Funções do Unity Catalog**| - Definidas e gerenciadas no Unity Catalog com recursos de segurança e conformidade integrados <br> - Cria um registro central para ferramentas que podem ser governadas como outros objetos do Unity Catalog <br> - Concede maior facilidade de descoberta e reutilização <br> - Ideal para aplicar transformações e agregações em grandes conjuntos de dados|
|**Ferramentas de código de agente**| - Definidas no código do agente de IA <br> - Úteis para chamar APIs REST, usar código arbitrário ou executar ferramentas de baixa latência <br> - Não possuem governança integrada e facilidade de descoberta de funções|

## a. Executando processamentos arbitrários

Ferramentas podem ser muito úteis para definir as tarefas que um agente pode executar. Muitas vezes, essas tarefas são específicas do nosso negócio e precisamos definir como o agente irá executá-las. Nesse casos, podemos usar uma lógica arbitrária para desenvolver essas rotinas de forma mais flexível.

Alguns casos de uso são:
* Cálculos matemáticos
* Tratamentos de texto
* Aplicação de regras de negócio
* Validações

In [0]:
%sql CREATE OR REPLACE FUNCTION valida_cpf(
  cpf STRING COMMENT 'Número do CPF'
)
RETURNS BIGINT
LANGUAGE PYTHON
COMMENT 'Use esta função para validar um CPF e convertê-lo para número. Retorna -1 se o CPF for inválido.'
AS
$$
  cpf = cpf.replace(".", "").replace("-", "")
  if len(cpf) != 11:
    return False
  elif not cpf.isdigit():
    return False
  else:
    d1 = (int(cpf[0])*1 + int(cpf[1])*2 + int(cpf[2])*3 + int(cpf[3])*4 + int(cpf[4])*5 + int(cpf[5])*6 + int(cpf[6])*7 + int(cpf[7])*8 + int(cpf[8])*9) % 11 % 10
    d2 = (int(cpf[0])*0 + int(cpf[1])*1 + int(cpf[2])*2 + int(cpf[3])*3 + int(cpf[4])*4 + int(cpf[5])*5 + int(cpf[6])*6 + int(cpf[7])*7 + int(cpf[8])*8 + d1*9) % 11 % 10
    if d1 == int(cpf[9]) and d2 == int(cpf[10]):
      return int(cpf)
    else:
      return -1
$$

In [0]:
%sql SELECT valida_cpf("111.111.111-11")

## b. Consultando dados estruturados

Para conseguirmos extrair o máximo de valor dos nossos agentes, precisamos que eles consigam acessar nossos dados corporativos. Somente com essa combinação conseguiremos de fato criar aplicações capazes de impactar o nosso negócio.

### Acessando dados do Lakehouse

Vamos ver como acessar os dados das nossas tabelas Delta.

In [0]:
%sql CREATE OR REPLACE FUNCTION consultar_cliente(id BIGINT)
RETURNS TABLE (id_cliente BIGINT, nome STRING, sobrenome STRING, num_pedidos INT)
COMMENT 'Use esta função para consultar os dados de um cliente'
RETURN SELECT id_cliente, nome, sobrenome, num_pedidos FROM academy.playground.clientes c WHERE c.id_cliente = consultar_cliente.id

In [0]:
%sql SELECT * FROM consultar_cliente(11111111111)


<img src="https://docs.databricks.com/aws/en/assets/images/create-online-table-473e834357fdf2f576706b4a9100850f.png" style="float: right; width: 800px; margin-top: 70px; margin-left: 10px">

### Lakebase

Para cenários onde necessitamos de menores latências, também podemos utilizar **[Databricks Lakebase](https://docs.databricks.com/aws/en/oltp/)**, que são uma cópia somente de leitura de uma tabela Delta que é armazenada em formato orientado a linhas, otimizado para acesso **online**.

O Lakebase é totalmente **serverless** e escalam automaticamente a capacidade de throughput conforme a carga de solicitações, proporcionando baixa latência e alto throughput no acesso a dados de qualquer escala.

O Lakebase também fornece **integração** com o Mosaic AI Model Serving, Feature Serving e aplicações de geração aumentada por recuperação (RAG), onde são usadas para consultas rápidas de dados.

## c. Consultando dados não-estruturados

<img src="https://www.databricks.com/sites/default/files/2024-01/db-vector-search-image-01_0.png?v=1705100714" style="float: right; width: 800px; margin-left: 10px">

No entanto, muitas vezes os dados que precisamos acessar não são necessariamente estruturados ou não estamos querendo fazer uma busca exata.

O **[Databricks Vector Search](https://docs.databricks.com/aws/en/generative-ai/vector-search)** é um banco de dados vetorial serverless, **integrado** de forma transparente na Data Intelligence Platform.

Ao contrário de outros bancos de dados, o Databricks Vector Search suporta a **sincronização automática** de dados da fonte para o índice, eliminando a manutenção complexa e cara de pipelines.

Ele aproveita as mesmas ferramentas de **segurança e governança** de dados que as organizações já construíram para maior tranquilidade.

Com seu design serverless, o Databricks Vector Search **escala** facilmente para suportar bilhões de embeddings e milhares de consultas em tempo real por segundo.

### i. Criando um Vector Search endpoint

Para criar o endpoint, siga os passos abaixo:

1. No **menu principal** à esquerda, clique em **Compute**
1. Na parte superior, clique na aba **Vector Search**
1. No canto superior direito, clique em **Create endpoint**
1. Digite o nome `academy-vs-endpoint`
1. Clique em **Confirm**

### ii. Consultando produtos similares

Outro cenário interessante também é o de pesquisas por similaridade. 

Por exemplo, um usuário pode estar buscando alguma característica específica do produto que não esteja categorizada nos filtros existentes do nosso website.

Dessa forma, podemos pesquisar dentro das descrições dos produtos e encontrar aqueles com maior relevância para o cliente, facilitando a descoberta destes produtos e aumentando as chances de conversão.

Para criar o índice no Vector Search, siga os passos abaixo:

1. No **menu principal** à esquerda, clique em **Catalog**
1. No canto superior esquerdo, pesquise a tabela `produtos`
1. No canto superior direito, clique em **Create** > **Vector search index**
1. Preencha as seguintes informações
    - **Name:** produtos_index
    - **Primary key:** id
    - **Embedding source column:** produto
    - **Embedding model:** databricks-gte-large-en
    - **Vector search endpoint:** academy-vs-endpoint
1. Clique em **Create**

In [0]:
%sql CREATE OR REPLACE FUNCTION buscar_produtos_semelhantes(descricao STRING)
RETURNS TABLE(id LONG, produto STRING, descricao STRING, search_score DOUBLE)
COMMENT 'Esta função recebe a descrição de um produto, que é utilizada para buscar produtos semelhantes'
RETURN SELECT * FROM vector_search(
  index => 'academy.playground.produtos_index',
  query => buscar_produtos_semelhantes.descricao,
  num_results => 10)
WHERE search_score BETWEEN 0.003 AND 0.99
LIMIT 3

In [0]:
%sql SELECT * FROM buscar_produtos_semelhantes("O fone de ouvido DEF é um dispositivo de áudio projetado para fornecer uma experiência de som imersiva e de alta qualidade. Com drivers de alta fidelidade e tecnologia de cancelamento de ruído, ele permite que você se perca na música ou nos detalhes de um podcast sem distrações. Além disso, seu design ergonômico garante confort durante o uso prolongado.")

## d. Usando prompt engineering

Para customizar o comportamento dos nossos modelos de IA Generativa, podemos utilizar prompts curados por especialistas. Dessa forma, conseguimos:

* Aproveitar melhor o conhecimento de especialistas em IA ou determinado domínio de negócio para aumentar a **eficiência** dos agentes
* Promover a **reutilização** desses ativos entre projetos 
* **Democratizar** o acesso a IA para usuários menos avançados

### Personalização de respostas

Com todas as informações extraídas, podemos aproveitá-las para gerar sugestões de respostas personalizadas para acelerar o trabalho dos nossos times de atendimento.

Outro ponto interessante é que, nesse processo, podemos aproveitar outras **informações estruturadas** que já tenhamos no nosso ambiente, como dados demográficos, psicográficos e o histórico de compras, para customizar ainda mais nossas respostas!

Vamos ver como fazer isso!

In [0]:
%sql CREATE OR REPLACE FUNCTION gerar_resposta(nome STRING, sobrenome STRING, num_pedidos INT, produto STRING, motivo STRING)
RETURNS TABLE(resposta STRING)
COMMENT 'Caso o cliente demonstre insatisfação com algum produto, use esta função para gerar uma resposta personalizada'
RETURN SELECT AI_QUERY(
    'databricks-gpt-oss-120b',
    CONCAT(
        "Você é um assistente virtual de um e-commerce. Nosso cliente, ", gerar_resposta.nome, " ", gerar_resposta.sobrenome, " que comprou ", gerar_resposta.num_pedidos, " produtos este ano estava insatisfeito com o produto ", gerar_resposta.produto, 
        ", pois ", gerar_resposta.motivo, ". Forneça uma breve mensagem empática para o cliente incluindo a oferta de troca do produto, caso  esteja em conformidade com a nossa política de trocas. A troca pode ser feita diretamente por esse assistente. ",
        "Eu quero recuperar sua confiança e evitar que ele deixe de ser nosso cliente. ",
        "Escreva uma mensagem com poucas sentenças. ",
        "Não adicione nenhum texto além da mensagem. ",
        "Não adicione nenhuma assinatura."
    )
)

In [0]:
%sql SELECT * FROM gerar_resposta("João", "Silva", 23, "tablet DEF", "duração da bateria")

# Exercício 03 - Criando o agente

Para decidir qual o melhor modelo e instrução para o nosso caso de uso, podemos utilizar o **[AI Playground](https://docs.databricks.com/en/large-language-models/ai-playground.html)**.

Assim, podemos testar rapidamente diversas combinações de modelos e instruções através de uma interface intuitiva e escolher a melhor opção par utilizarmos no nosso projeto.

## a. Configurando o Agente

<img src="https://docs.databricks.com/en/_images/ai-playground.gif" style="float: right; margin-top:20px; margin-bottom:50px; padding-left: 10px" width=600>

Siga os passos abaixo:

1. No **menu principal** à esquerda, clique em **Playground**
1. Clique no **seletor de modelos** e selecione o modelo **GPT OSS 120B**
1. Clique no ícone **Add endpoint**
1. Adicione a instrução abaixo:<br>
    `Você é um assistente virtual de um e-commerce. Para responder à perguntas, é necessário que o cliente forneça um CPF válido. Caso ainda não tenha essa informação, solicite o CPF educadamente. Após validar o CPF, lembre-se de consultar os dados do cliente para personalizar suas respostas. Caso o CPF do cliente não exista na nossa base, peça educadamente um novo CPF. Você pode responder perguntas sobre entrega, devolução de produtos, status de pedidos, entre outros. Se você não souber como responder a pergunta, diga que você não sabe. Não invente ou especule sobre nada. Sempre que perguntado sobre procedimentos, consulte nossa base de conhecimento.`
    <br>
1. Clique em **Tools** > **Add tool**
1. Digite o caminho das suas **ferramentas**: 
    - `academy.playground.valida_cpf`
    - `academy.playground.consultar_cliente`
    - `academy.playground.buscar_produtos_semelhantes`
    - `academy.playground.gerar_resposta`
1. Clique no ícone **enviar**

## b. Testando o Agente

Agora, podemos avaliar as respostas, o tempo e o custo do nosso agente para entender se ele atende às necessidades do nosso projeto!

Envie as mensagens abaixo para o seu agente:
1. Oi!
1. Meu CPF é 111.111.111-11
1. Comprei um tablet DEF, mas a duração da bateria é muito curta
1. Poderia sugerir um produto melhor?

# Exercício 04 - Avaliando e Implantando Agentes

Depois de prototipar nosso agente, podemos mover para as etapas de avaliação e implantação.

## a. Avaliação do agente

Antes de colocar qualquer agente em produção, assim como qualquer outro tipo de software, é extremamente importante avaliar a sua qualidade. Para automatizar esse processo, podemos usar os **[built-in AI Judges](https://docs.databricks.com/aws/en/mlflow3/genai/eval-monitor/concepts/judges/pre-built-judges-scorers)**. Desta forma, temos acesso a avaliadores para tarefas comuns da validação de qualidade desenvolvidos pelo time de pesquisa da Databricks. 

Siga os passos abaixo para avaliar o agente:
1. Na parte central superior do Playground, clique em **Get code** > **Create agent notebook**
1. No notebook criado, execute as células das sessões abaixo para criar o agente e observar suas respostas
    - Prerequisites
    - Define the agent in code
    - Test the agent
1. Na sessão **Log the agent as an MLflow model**, adicione o índice do Vector Search e a tabela de clientes aos recursos do agente
```
from mlflow.models.resources import DatabricksTable, DatabricksVectorSearchIndex
resources.append(DatabricksTable('academy.playground.clientes'))
resources.append(DatabricksVectorSearchIndex('academy.playground.produtos_index'))
```
4. Execute as células da sessão **Evaluate the agent with Agent Evaluation** para avaliar a performance do agente usando AI Judges
1. Clique em **View evaluation results** para analisar o resultado da avaliação
1. Execute as células da sessão **Perform pre-deployment validation of the agent** para validar a execução do agente a partir do Mlflow

## b. Implantando o agente como uma API REST

Agora, podemos mover nosso agente para produção. Para permitir uma fácil integração com diversos sistemas e a execução em tempo real do agente, iremos implantá-lo como uma API REST.

Para facilitar esse processo, iremos utilizar:
  - **Unity Catalog** para versionar e controlar o acesso aos artefatos do agente
  - **[Model Serving](https://docs.databricks.com/aws/en/machine-learning/model-serving/)** para criar um endpoint serverless para o agente

Siga os passos abaixo para implantar o agente:
1. Na sessão **Register the model to Unity Catalog**:
    - Preencha as seguintes variáveis:
        - **catalog:** academy
        - **schema:** playground
        - **model_name:** agente_atendimento
    - Execute as células para registrar o agente no Unity Catalog
1. Na sessão **Deploy the agent**, habilite a escalabilidade para zero, conforme abaixo, e execute as células para implantar o agente como uma API REST
```
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, scale_to_zero=True, tags = {"endpointSource": "playground"})
```
3. Acesse o link gerado ao lado de **View status** para visualizar o endpoint criado
1. Após o status do endpoint mudar para **Ready**, clique no botão **Use** para conversar com o seu agente